In [14]:
import keras
from keras import layers
import random
import sys
import numpy as np

In [2]:
# Opening a txt file of Mary Shelley's Frankenstein from https://www.gutenberg.org/ebooks/84

text = open("Frankenstein Text.txt","r", encoding="utf-8").read().lower()
print('Corpus length:', len(text))

Corpus length: 437330


In [4]:
maxlen = 60

step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

Number of sequences: 145757


In [5]:
chars = sorted(list(set(text)))
print('Unique Characters:', len(chars))

Unique Characters: 67


In [6]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [8]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [10]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [12]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [13]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
for epoch in range(1, 30):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: ', generated_text)
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
                    
                

epoch 1
1139/1139 [==============================] - 77s 65ms/step - loss: 2.0215
--- Generating with seed:  ppiness who did not enjoy this blessing.

“i agree with you,
----- temperature: 0.2
ppiness who did not enjoy this blessing.

“i agree with you, and the most in the sides of the fire the secting the section of the section which i was the section of the man was some the section of the saw he have a sould a comman and my have and the section of the such i sould commine the sight and the saw the saw the sees the such of the sections and the section of the section of the sears the sees i was secred to her the section of his father the secting
----- temperature: 0.5
es i was secred to her the section of his father the secting i so hearness of the was i might i have freen i might was endeary she finglank of a covery ever whem the pams and a sundering to he undersion to entained my strong the sempless of the said her, and i sool the sermed the unfire at but him the fires i was the mest